In [1]:
using DrWatson
@quickactivate "SNSensitivityEstimate"

Precompiling DrWatson
  ✓ JLD2
  ✓ DrWatson
  2 dependencies successfully precompiled in 27 seconds. 24 already precompiled.
[ Info: Precompiling DrWatson [634d3b9d-ee7a-5ddf-bec9-22491ea816e1]
┌ Warning: Module PrecompileTools with build ID fafbfcfd-04bd-c4d2-0007-f9c517e90c82 is missing from the cache.
│ This may mean PrecompileTools [aea7be01-6a6a-4083-8856-8a6e6704d82a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing DrWatson [634d3b9d-ee7a-5ddf-bec9-22491ea816e1].
[ Info: Precompiling JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
┌ Warning: Module PrecompileTools with build ID fafbfcfd-04bd-c4d2-0007-f9c517e90c82 is missing from the cache.
│ This may mean PrecompileTools [aea7be01-6a6a-4083-8856-8a6e6704d82a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Im

In [2]:
push!(LOAD_PATH, srcdir())

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/sps/nemo/scratch/mpetro/Projects/PhD/SNSensitivityEstimate/src"

In [3]:
using SensitivityModule

[ Info: Precompiling SensitivityModule [top-level]
┌ Warning: Module PrecompileTools with build ID fafbfcfd-04bd-c4d2-0007-f9c517e90c82 is missing from the cache.
│ This may mean PrecompileTools [aea7be01-6a6a-4083-8856-8a6e6704d82a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing SensitivityModule [top-level].
Precompiling StatsPlots
  ✓ Measures
  ✓ fzf_jll
  ✓ EpollShim_jll
  ✓ Unzip
  ✓ TableOperations
  ✓ Xorg_libICE_jll
  ✓ Wayland_protocols_jll
  ✓ mtdev_jll
  ✓ gperf_jll
  ✓ Arpack_jll
  ✓ Dbus_jll
  ✓ libevdev_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libXfixes_jll
  ✓ Latexify
  ✓ Distances
  ✓ Widgets
  ✓ JLFzf
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXrandr_jll
  ✓ Wayland_jll
  ✓ Xorg_libSM_jll
  ✓ FFMPEG
  ✓ eudev_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Arpack
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_xcb_util_image_jll

In [4]:
using CSV, DataFramesMeta, CairoMakie, StatsBase, Measurements;

In [5]:
function get_best_sens(signal, dir)
    df = DataFrame(thalf = [], roi = [], eff = [], b = [], x0 = [])
    for f in readdir(dir)
        if occursin(signal, f)
            try
                d = CSV.File(joinpath(dir, f)) |> DataFrame
                append!(df, d, promote = true)
            catch 
                @warn "invalid file: $f"
            end
        end
    end
    
    best = argmax(df.thalf)

    return DataFrame(signal = signal, thalf = df.thalf[best], roi = df.roi[best], eff = df.eff[best], b = df.b[best])
end

function get_best_sens!(df, signal, dir)
    d = get_best_sens(signal, dir)
    append!(df, d, promote =true)
end

get_best_sens! (generic function with 1 method)

In [6]:
df = DataFrame(thalf = [], roi = [], eff = [], b = [], x0 = [])
signal = "bb0nu_foil_bulk"

"bb0nu_foil_bulk"

In [7]:
df_best = get_best_sens(signal, "results/phi_esum_emax_r")

Row,signal,thalf,roi,eff,b
,String,Float64,String,Float64,String31
1,bb0nu_foil_bulk,3.95703e24,"(phi = (12.0, 162.0), sumE = (2726.0, 3499.0), maxE = (1128.0, 2831.0), r = (0.0, 80.0))",0.127108,0.4999 ± 0.006


In [8]:
for s in ["bb0nuM1_foil_bulk", "bb0nuM2_foil_bulk", "RH037_foil_bulk"]
    get_best_sens!(df_best, s, "results/phi_esum_emax_r")
end

In [9]:
df_best

Row,signal,thalf,roi,eff,b
,String,Any,String,Any,String31
1,bb0nu_foil_bulk,3.95703e24,"(phi = (12.0, 162.0), sumE = (2726.0, 3499.0), maxE = (1128.0, 2831.0), r = (0.0, 80.0))",0.127108,0.4999 ± 0.006
2,bb0nuM1_foil_bulk,1.45998e23,"(phi = (17.0, 171.0), sumE = (2519.0, 3479.0), maxE = (1133.0, 3410.0), r = (1.0, 67.0))",0.0146559,19.53 ± 0.34
3,bb0nuM2_foil_bulk,2.29577e22,"(phi = (15.0, 177.0), sumE = (961.0, 2687.0), maxE = (622.0, 2834.0), r = (1.0, 99.0))",0.105379,61740.0 ± 1000.0
4,RH037_foil_bulk,9.986115753367956e21 ± 0.0,"(phi = (31.0, 170.0), sumE = (325.0, 2387.0), maxE = (285.0, 2199.0), r = (15.0, 75.0))",0.04876854 ± 0.0,69890.0 ± 970.0


| **signal**        | **ROI**             | **bkg counts in ROI** | **efficiency** | **t12** |
|------------------:|--------------------:|----------------------:|---------------:|--------:|
| bb0nu\_foil\_bulk | 2700.0 - 3200.0 keV | 1.03±0.09             | 1.5            | 4.14e25 |
| RH037\_foil\_bulk | 300.0 - 2900.0 keV | 114892.0±79.0         | 0.08           | 1.3e22  |


bb0nu
```julia
(
    phi = (10.0, 179.0), 
    sumE = (2710.0, 3348.0), 
    r = (0.0, 90.0)), 
    4.224324764336815e24, 
    0.14519695, 
    0.7282654628444958
)
```

RH
```julia
((phi = (2.0, 175.0), sumE = (95.0, 3003.0), r = (1.0, 80.0)), 1.2919311994819359e22, 0.08111526, 115520.49989921192)
```